In [1]:
using Pkg
Pkg.add("BenchmarkTools")
using BenchmarkTools
'''
using PyPlot

#use
x = linspace(0,2*pi,1000) #in Julia 0.6
x = range(0; stop=2*pi, length=1000); y = sin.(3 * x + 4 * cos.(2 * x));
plot(x, y, color="red", linewidth=2.0, linestyle="--")
title("A sinusoidally modulated sinusoid")
'''

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  99.9 %0.0 %Fetching: [=====>                                   ]  10.4 %Fetching: [=====>                                   ]  11.2 %Fetching: [======>                                  ]  12.7 %Fetching: [======>                                  ]  14.2 %Fetching: [=======>                                 ]  16.1 %Fetching: [========>                                ]  17.7 %Fetching: [========>                                ]  18.3 %Fetching: [========>                                ]  18.8 %Fetching: [========>                                ]  19.4 %Fetching: [========>                                ]  19.9 %Fetching: [=============>                           ]  30.4 %Fetching: [=============>                           ]  31.2 %Fetching: [=============>                           ]  31.7 %Fetching: [========

  Building TimeZones → `~/.julia/packages/TimeZones/A5RcB/deps/build.log`
  Building FFTW ─────→ `~/.julia/packages/FFTW/MJ7kl/deps/build.log`


┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1192
┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1192
ERROR: LoadError: LoadError: PyCall not properly installed. Please run Pkg.build("PyCall")
Stacktrace:
 [1] error(::String) at ./error.jl:33
 [2] top-level scope at /Users/rjjarvis/.julia/packages/PyCall/RQjD7/src/startup.jl:44 [inlined]
 [3] top-level scope at ./none:0
 [4] include at ./boot.jl:317 [inlined]
 [5] include_relative(::Module, ::String) at ./loading.jl:1044
 [6] include at ./sysimg.jl:29 [inlined]
 [7] include(::String) at /Users/rjjarvis/.julia/packages/PyCall/RQjD7/src/PyCall.jl:1
 [8] top-level scope at none:0
 [9] include at ./boot.jl:317 [inlined]
 [10] include_relative(::Module, ::String) at ./loading.jl:1044
 [11] include(::Module, ::String) at ./sysimg.jl:29
 [12] top-level scope at none:2
 [13] eval at ./boot.jl:319 [inlined]
 [14] eval(::Expr) at ./client.jl:393
 [15] top-l

ErrorException: Failed to precompile PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee] to /Users/rjjarvis/.julia/compiled/v1.0/PyPlot/oatAj.ji.

Set up the environment.

In [2]:
gui()
using Pkg
using IJulia


using Knet;
using DataFrames
using Plots;
using StatsPlots;

include(Knet.dir("data","housing.jl"));
x,y = housing();

UndefVarError: UndefVarError: gui not defined

# Which features are we plotting against

In [3]:


#Now that we have the data
predict(ω, x) = ω[1] * x .+ ω[2];
loss(ω, x, y) = mean(abs2, predict(ω, x)-y);
lossgradient = grad(loss);

function train(ω, data; lr=0.01)
    for (x,y) in data
        dω = lossgradient(ω, x, y)
        for i in 1:length(ω)
            ω[i] -= dω[i]*lr
        end
    end
    return ω
end;


gr();
p0 = scatter(x', y[1,:], layout=(3,5), reg=true, size=(950,500))

ω = Any[ 0.1*randn(1,13), 0.0 ];
errdf = DataFrame(Epoch=1:20, Error=0.0);
let
    cntr=1
    cntr::Int64
    println(cntr)
    for i=1:200
        train(ω, [(x,y)])
        if mod(i, 10) == 0
            println("Epoch $i: $(round(loss(ω,x,y)))")
            errdf[cntr, :Epoch]=i
            errdf[cntr, :Error]=loss(ω,x,y)
            cntr+=1
        end
    end
end


UndefVarError: UndefVarError: grad not defined

In [ ]:
plot(p0, layout=(1,1), size=(950,500))

In [4]:

p1 = scatter(errdf[:,:Epoch], errdf[:,:Error], xlabel="Epoch", ylabel="Error")
p2 = scatter(predict(ω, x)', y', reg=true, xlabel="Predicted", ylabel="Observed")
plot(p1, p2, layout=(1,2), size=(950,500))

UndefVarError: UndefVarError: errdf not defined

In [5]:


ω = Any[0.1f0*randn(Float32,64,13), zeros(Float32,64,1),
        0.1f0*randn(Float32,15,64), zeros(Float32,15,1),
        0.1f0*randn(Float32,1,15),  zeros(Float32,1,1)]
function predict(ω, x)
    x = mat(x)
    for i=1:2:length(ω)-2
        x = relu.(ω[i]*x .+ ω[i+1])
    end
    return ω[end-1]*x .+ ω[end]
end

loss(ω, x, y) = mean(abs2, predict(ω, x)-y)
lossgradient = grad(loss)

errdf = DataFrame(Epoch=1:60, Error=0.0)
let
    cntr = 1
    for i=1:600
        train(ω, [(x,y)])
        if mod(i, 10) == 0
            errdf[cntr, :Epoch]=i
            errdf[cntr, :Error]=loss(ω,x,y)
            cntr+=1
        end
    end
end



p3 = scatter(errdf[:,:Epoch], errdf[:,:Error], xlabel="Epoch", ylabel="Error")
p4 = scatter(predict(ω, x)', y', reg=true, xlabel="Predicted", ylabel="Observed")
# layout=(1,2), size=(950,500))
plot(p3, p4,layout=(1,2), size=(950,500))
# layout=(1,2), size=(950,500))#,png)
plot(p2, p4,layout=(1,2), size=(950,500))


xtrn, xtst = x[:, 1:400], x[:, 401:end]
ytrn, ytst = y[:, 1:400], y[:, 401:end]

ω = Any[0.1f0*randn(Float32,64,13), zeros(Float32,64,1),
        0.1f0*randn(Float32,15,64), zeros(Float32,15,1),
        0.1f0*randn(Float32,1,15),  zeros(Float32,1,1)]
errdf = DataFrame(Epoch=1:60, TrnError=0.0, ValError=0.0)
let
    cntr = 1
    for i=1:600
        train(ω, [(xtrn, ytrn)])
        if mod(i, 10) == 0
            errdf[cntr, :Epoch]=i
            errdf[cntr, :TrnError]=loss(ω,xtrn,ytrn)
            errdf[cntr, :ValError]=loss(ω,xtst,ytst)
            #print(cntr,errdf)#TrnError)
            cntr+=1
        end
    end
end

#using StatPlots
@df errdf[5:60,:] plot(:Epoch, [:ValError, :TrnError], xlabel="Epoch", ylabel="Error",
                       label=["Validation" "Training"], lw=3)

#After doing

#predict
#Epoch 10:


UndefVarError: UndefVarError: grad not defined

The data Frame below has columns, meaning:
* Training Error,
* Network training Epoch
* Network value error.

In [6]:
errdf

UndefVarError: UndefVarError: errdf not defined